In [79]:
import os
import sys
sys.path.append(os.path.join(os.getcwd(), '..'))
import time
import torch
import warnings
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import math

from CoExBO._utils import TensorManager
from CoExBO._coexbo import CoExBOwithSimulation, AlphaPIBOwithSimulation, CoTrustBOwithSimulation, StateManager
from CoExBO._prior import Uniform


tm = TensorManager()
warnings.filterwarnings('ignore')

# Synthetic Functions

In [80]:
def BraninFunction(x):
    x = torch.atleast_2d(x)
    return ((x.sin() + (3*x).cos()/2).square() / ((x/2).square()+0.3)).prod(axis=1).double()

def Ackley4Function(x, a=20.0, b=0.2, c=2 * math.pi):
    """
    Vectorized negated Ackley function for maximization.
    x: torch tensor of shape (..., 4)
    Returns: torch tensor of shape (...)
    """
    d = 4 #x.shape[-1]

    sum_sq = torch.sum(x**2, dim=-1) / d
    sum_cos = torch.sum(torch.cos(c * x), dim=-1) / d

    term1 = -a * torch.exp(-b * torch.sqrt(sum_sq))
    term2 = -torch.exp(sum_cos)
    f = term1 + term2 + a + math.e

    return -f.double()

def HolderTableFunction(x):
    """
    Torch-native vectorized negated Holder Table function for maximization.
    Input:
        x: torch.Tensor of shape (..., 2)
    Output:
        torch.Tensor of shape (...)
    """
    x = x.double()
    x1, x2 = x[..., 0], x[..., 1]
    r = torch.sqrt(x1**2 + x2**2)
    value = -torch.abs(torch.sin(x1) * torch.cos(x2) * torch.exp(torch.abs(1 - r / math.pi)))
    return -value.double()

def MichalewiczFunction(x, m=10):
    """
    Torch-native Michalewicz function (negated for maximization).
    Input: x (tensor of shape [..., d])
    Output: tensor of shape [...]
    """
    x = x.double()
    d = x.shape[-1]
    i = torch.arange(1, d + 1, dtype=x.dtype, device=x.device).view(1, -1)
    value = -torch.sum(torch.sin(x) * torch.sin(i * x**2 / math.pi)**(2 * m), dim=-1)
    return -value  # negate for maximization

def RosenbrockFunction(x):
    """
    Torch-native Rosenbrock function (negated for maximization).
    Input: x (tensor of shape [..., d])
    Output: tensor of shape [...]
    """
    x = x.double()
    value = torch.sum(100 * (x[..., 1:] - x[..., :-1]**2)**2 + (1 - x[..., :-1])**2, dim=-1)
    return -value  # negate for maximization

def Hartmann6Function(x):
    """
    Torch-native 6D Hartmann function (already a maximization function).
    Input: x (tensor of shape [..., 6])
    Output: tensor of shape [...]
    """
    x = x.double()

    alpha = torch.tensor([1.0, 1.2, 3.0, 3.2], dtype=x.dtype, device=x.device)
    A = torch.tensor([
        [10, 3, 17, 3.5, 1.7, 8],
        [0.05, 10, 17, 0.1, 8, 14],
        [3, 3.5, 1.7, 10, 17, 8],
        [17, 8, 0.05, 10, 0.1, 14],
    ], dtype=x.dtype, device=x.device)
    P = 1e-4 * torch.tensor([
        [1312, 1696, 5569, 124, 8283, 5886],
        [2329, 4135, 8307, 3736, 1004, 9991],
        [2348, 1451, 3522, 2883, 3047, 6650],
        [4047, 8828, 8732, 5743, 1091, 381],
    ], dtype=x.dtype, device=x.device)

    # x: [..., 6] → [batch, 1, 6]
    x_expanded = x.unsqueeze(-2)  # [..., 1, 6]
    inner = torch.sum(A.unsqueeze(1) * (x_expanded - P.unsqueeze(1))**2, dim=-1)  # [4, ...,]
    value = torch.sum(alpha.view(-1, 1) * torch.exp(-inner), dim=0)
    return value  # already maximization

def StyblinskyFunction(x):
    """
    Torch-native Styblinski–Tang function (negated for maximization).
    Input: x (tensor of shape [..., d])
    Output: tensor of shape [...]
    """
    x = x.double()
    value = 0.5 * torch.sum(x**4 - 16 * x**2 + 5 * x, dim=-1)
    return -value 

In [81]:
# Assume all function definitions (e.g. ackley_4d, holder_table, etc.) are already defined

def define_function(function_name):
    global lower_limit, upper_limit, ground_truth, n_dims, bounds, domain, true_function, samples, true_max

    if function_name.lower() == "branin":
        lower_limit = -2
        upper_limit = 3
        ground_truth = torch.tensor([-1.02543108, -1.02543108])
        n_dims = 2

        mins = lower_limit * torch.ones(n_dims)
        maxs = upper_limit * torch.ones(n_dims)
        bounds = torch.vstack([mins, maxs])

        domain = Uniform(bounds)
        true_function = BraninFunction
        true_max = true_function(ground_truth)

    elif function_name.lower() == "ackley":
        lower_limit = -32.768
        upper_limit = 32.768
        ground_truth = torch.tensor([0.0, 0.0, 0.0, 0.0])
        n_dims = 4

        mins = lower_limit * torch.ones(n_dims)
        maxs = upper_limit * torch.ones(n_dims)
        bounds = torch.vstack([mins, maxs])

        domain = Uniform(bounds)
        true_function = Ackley4Function
        true_max = true_function(ground_truth)


    elif function_name.lower() == "holder":
        lower_limit = -10
        upper_limit = 10
        ground_truth = torch.tensor([8.05502, 9.66459])
        n_dims = 2

        mins = lower_limit * torch.ones(n_dims)
        maxs = upper_limit * torch.ones(n_dims)
        bounds = torch.vstack([mins, maxs])

        domain = Uniform(bounds)
        true_function = HolderTableFunction
        true_max = true_function(ground_truth)


    elif function_name.lower() == "michalewicz":
        lower_limit = 0
        upper_limit = np.pi
        n_dims = 5
        ground_truth = torch.tensor([2.20, 1.57, 1.29, 1.40, 1.52])  # example near-optimum

        mins = lower_limit * torch.ones(n_dims)
        maxs = upper_limit * torch.ones(n_dims)
        bounds = torch.vstack([mins, maxs])

        domain = Uniform(bounds)
        true_function = MichaleWiczFunction
        true_max = true_function(ground_truth)


    elif function_name.lower() == "rosenbrock":
        lower_limit = -5
        upper_limit = 10
        n_dims = 3
        ground_truth = torch.ones(n_dims)
        
        mins = lower_limit * torch.ones(n_dims)
        maxs = upper_limit * torch.ones(n_dims)
        bounds = torch.vstack([mins, maxs])

        domain = Uniform(bounds)
        true_function = RosenbrockFunction
        true_max = true_function(ground_truth)


    elif function_name.lower() == "hartmann":
        lower_limit = 0
        upper_limit = 1
        n_dims = 6
        ground_truth = torch.tensor([0.20169, 0.15001, 0.47687, 0.27533, 0.31165, 0.6573])

        mins = lower_limit * torch.ones(n_dims)
        maxs = upper_limit * torch.ones(n_dims)
        bounds = torch.vstack([mins, maxs])

        domain = Uniform(bounds)
        true_function = Hartmann6Function
        true_max = true_function(ground_truth)


    elif function_name.lower() == "styblinski-tang":
        lower_limit = -5
        upper_limit = 5
        n_dims = 4
        ground_truth = torch.full((n_dims,), -2.903534)

        mins = lower_limit * torch.ones(n_dims)
        maxs = upper_limit * torch.ones(n_dims)
        bounds = torch.vstack([mins, maxs])

        domain = Uniform(bounds)
        true_function = StyblinskyFunction
        true_max = true_function(ground_truth)


    else:
        raise ValueError(f"Unknown function: {function_name}")

    # Sample the domain
    samples = domain.sample(1000)


# Setup

In [82]:
colour_map = 'summer'
resolution = 200
n_init_pref = 100      # number of initial random samples for preferential learning
n_init_obj = 20        # number of initial random samples for objective function
def compute_regret(dataset_obj, true_optimum_value):
    best_observed = dataset_obj[1].max().item()
    return abs(true_optimum_value - best_observed)

In [83]:
function_name = 'branin'
define_function(function_name)

# CoExBO

In [69]:
sigma = 0.1
n_iterations = 100     # number of iterations
seed = 0              # random seed for reproduce the results
log_regrets = np.zeros(n_iterations)
torch.manual_seed(seed)
experiment_name = 'Name'

In [70]:
coexbo = CoExBOwithSimulation(domain, true_function, sigma=sigma)
dataset_obj, dataset_duel = coexbo.initial_sampling(n_init_obj, n_init_pref)

state = StateManager(n_dims=n_dims)

In [78]:
results = []
for t in range(n_iterations):
    beta, gamma = state(t)
    result, dataset_obj, dataset_duel = coexbo(
        dataset_obj, dataset_duel, beta, gamma,
    )
    print(f"{len(dataset_obj[0])}) Best value: {dataset_obj[1].max().item():.5e}")
    results.append(result)
    regret = compute_regret(dataset_obj, true_max)
    log_regrets[t] = np.log10(regret + 1e-8)  # add epsilon to avoid log(0)

np.savez(experiment_name, log_regrets=log_regrets)

0) parameters: beta 5.657e-01 gamma 5.000e-01
31) Best value: 1.06041e+01
1) parameters: beta 8.000e-01 gamma 5.000e-01
32) Best value: 1.06041e+01
2) parameters: beta 9.798e-01 gamma 5.000e-01
33) Best value: 1.06041e+01
3) parameters: beta 1.131e+00 gamma 5.000e-01
34) Best value: 1.06041e+01
4) parameters: beta 1.265e+00 gamma 5.000e-01
35) Best value: 1.06041e+01
5) parameters: beta 1.386e+00 gamma 5.000e-01
36) Best value: 1.06041e+01
6) parameters: beta 1.497e+00 gamma 5.000e-01
37) Best value: 1.06041e+01
7) parameters: beta 1.600e+00 gamma 5.000e-01


KeyboardInterrupt: 

# AlphaPiBO

In [75]:
alpha = 0.5
n_iterations = 10     # number of iterations
seed = 0              # random seed for reproduce the results
log_regrets = np.zeros(n_iterations)
# initial setting
torch.manual_seed(seed)
experiment_name = 'AllphaBO_baseline_branin.npz'

In [76]:
alphapibo = AlphaPIBOwithSimulation(domain, true_function, 
                                 sigma=alpha, uncertain_prior=True)
dataset_obj, dataset_duel = alphapibo.initial_sampling(n_init_obj, n_init_pref)

state = StateManager(n_dims=n_dims, gamma_init=alpha, probabilistic_pi="alphapi")

In [77]:
results = []
for t in range(n_iterations):
    beta, gamma = state(t)
    result, dataset_obj, dataset_duel = alphapibo(
        dataset_obj, dataset_duel, beta, alpha,
    )
    print(f"{len(dataset_obj[0])}) Best value: {dataset_obj[1].max().item():.5e}")
    results.append(result)
    regret = compute_regret(dataset_obj, true_max)
    log_regrets[t] = np.log10(regret + 1e-8)  # add epsilon to avoid log(0)

np.savez(experiment_name, log_regrets=log_regrets)

results = torch.tensor(results)

0) parameters: beta 5.657e-01 gamma 5.000e-01
21) Best value: 6.12073e+00
1) parameters: beta 8.000e-01 gamma 5.000e-01
22) Best value: 7.49731e+00
2) parameters: beta 9.798e-01 gamma 5.000e-01
23) Best value: 8.97936e+00
3) parameters: beta 1.131e+00 gamma 5.000e-01
24) Best value: 1.02320e+01
4) parameters: beta 1.265e+00 gamma 5.000e-01
25) Best value: 1.05683e+01
5) parameters: beta 1.386e+00 gamma 5.000e-01
26) Best value: 1.05930e+01
6) parameters: beta 1.497e+00 gamma 5.000e-01
27) Best value: 1.06024e+01
7) parameters: beta 1.600e+00 gamma 5.000e-01
28) Best value: 1.06029e+01
8) parameters: beta 1.697e+00 gamma 5.000e-01
29) Best value: 1.06029e+01
9) parameters: beta 1.789e+00 gamma 5.000e-01
30) Best value: 1.06029e+01


# PIBO

In [ ]:
sigma = 0.1
n_iterations = 100     # number of iterations
seed = 0              # random seed for reproduce the results
log_regrets = np.zeros(n_iterations)
torch.manual_seed(seed)
experiment_name = 'Name'

In [ ]:
coexbo = CoExBOwithSimulation(domain, true_function, sigma=sigma, probabilistic_pi=False)
dataset_obj, dataset_duel = coexbo.initial_sampling(n_init_obj, n_init_pref)

state = StateManager(n_dims=n_dims)

In [ ]:
results = []
for t in range(n_iterations):
    beta, gamma = state(t)
    result, dataset_obj, dataset_duel = coexbo(
        dataset_obj, dataset_duel, beta, gamma,
    )
    print(f"{len(dataset_obj[0])}) Best value: {dataset_obj[1].max().item():.5e}")
    results.append(result)
    regret = compute_regret(dataset_obj, true_max)
    log_regrets[t] = np.log10(regret + 1e-8)  # add epsilon to avoid log(0)

np.savez(experiment_name, log_regrets=log_regrets)

# TrustCoBO

TODO: Verify the arguments

In [65]:
alpha = 0.5
n_iterations = 100     # number of iterations
seed = 0              # random seed for reproduce the results
log_regrets = np.zeros(n_iterations)
torch.manual_seed(seed)
experiment_name = 'Name'



In [66]:
trustbo = CoTrustBOwithSimulation(domain, true_function, sigma=0.1)
dataset_obj, dataset_duel = trustbo.initial_sampling(n_init_obj, n_init_pref)
state = StateManager(n_dims=n_dims, gamma_init=alpha, probabilistic_pi="trustCo")

In [67]:
results = []
for t in range(n_iterations):
    beta, gamma = state(t)
    result, dataset_obj, dataset_duel = coexbo(
        dataset_obj, dataset_duel, beta, gamma,
    )
    print(f"{len(dataset_obj[0])}) Best value: {dataset_obj[1].max().item():.5e}")
    results.append(result)
    regret = compute_regret(dataset_obj, true_max)
    log_regrets[t] = np.log10(regret + 1e-8)  # add epsilon to avoid log(0)

np.savez(experiment_name, log_regrets=log_regrets)

AttributeError: 'NoneType' object has no attribute 'alpha'